In [1]:
'''An attempt to classify a recursive computation from a for loop implementation 
using only a state trace of the register state of the 6502 processor it's simulated on.'''
import random
import numpy as np
import glob
from scipy import misc

In [2]:
#Parameters
mode = 'sv'
trace_dir = 'traces'
image_dir = 'images'

In [3]:
#Clean up
!make clean 

rm -rf *~ *.o *.map *.img *.lst *.out memcpy.s memcpy backup 'images' *.s traces tmp


In [6]:
%%time 
#Generate some 6502 images
#Avoid rollover of the 8 bit regs by not hitting 256.
#Avoid 0 because gave weird trace.
for start in range(1,5):
    !make START={start} MAX={start+100} IMGDIR={image_dir}

../../datasources/6502/ext/install/bin/cc65 -D START=1 -D MAX=101 -D__6502__ -t none -O -Oi --cpu 6502 forCount.c
../../datasources/6502/ext/install/bin/ca65 --cpu 6502 forCount.s -l forCount.lst
../../datasources/6502/ext/install/bin/ld65 -o forCount -C ../../datasources/6502/apps/cc/apps/bu6502.cfg  forCount.o ../../datasources/6502/apps/cc/apps/bu6502.lib   -m forCount.map
[ -d 'backup' ] || mkdir backup
mv *.lst *.map *.s backup
[ -d images ] || mkdir images
mv forCount 1_101forCount.img
mv *.img images
rm forCount.o
../../datasources/6502/ext/install/bin/cc65 -D START=2 -D MAX=102 -D__6502__ -t none -O -Oi --cpu 6502 forCount.c
../../datasources/6502/ext/install/bin/ca65 --cpu 6502 forCount.s -l forCount.lst
../../datasources/6502/ext/install/bin/ld65 -o forCount -C ../../datasources/6502/apps/cc/apps/bu6502.cfg  forCount.o ../../datasources/6502/apps/cc/apps/bu6502.lib   -m forCount.map
[ -d 'backup' ] || mkdir backup
mv *.lst *.map *.s backup
[ -d images ] || mkdir images
mv for

In [7]:
%%time
#Run images through 6520 simulator to generate traces.
!mkdir -p {trace_dir}
traces = !ls {image_dir}
for t in traces:
    !../../datasources/6502/6502 -t {mode} -o {trace_dir}/{t}.trc {image_dir}/{t} /dev/null 2> /dev/null

CPU times: user 22.7 ms, sys: 20.3 ms, total: 43.1 ms
Wall time: 2.22 s
